In [2]:
import pandas as pd
import numpy as np

# Load the training and testing data
train_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\keystrokes.csv')
test_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\test1.csv')

# Verify column names
print(train_data.columns)
print(test_data.columns)

# Strip leading/trailing spaces from column names if necessary
train_data.columns = train_data.columns.str.strip()
test_data.columns = test_data.columns.str.strip()

# Add a new column named 'target' with the value 1 for all rows
train_data.insert(3, 'target', 1)
test_data.insert(3, 'target', 1)

# Assuming all data in train_data is from the owner (label=1) and test_data is from non-owner (label=0)
train_data['label'] = 1
test_data['label'] = 0

# Convert 'time' column to datetime, coercing errors
train_data['time'] = pd.to_datetime(train_data['time'], errors='coerce')
test_data['time'] = pd.to_datetime(test_data['time'], errors='coerce')

# Handle NaT values - you can either drop them or fill them with some value
train_data.dropna(subset=['time'], inplace=True)
test_data.dropna(subset=['time'], inplace=True)

# Ensure 'key' column exists after stripping spaces
if 'key' not in train_data.columns or 'key' not in test_data.columns:
    raise KeyError("The 'key' column is missing from the dataset after verifying column names.")

# Calculate additional features for training data
train_data['duration'] = train_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)
train_features = train_data.groupby('key').agg({
    'duration': ['mean', 'std'],
    'time': ['count']
}).reset_index()
train_features.columns = ['_'.join(col).strip() for col in train_features.columns.values]
train_features.fillna(0, inplace=True)

# Calculate additional features for testing data
test_data['duration'] = test_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)
test_features = test_data.groupby('key').agg({
    'duration': ['mean', 'std'],
    'time': ['count']
}).reset_index()
test_features.columns = ['_'.join(col).strip() for col in test_features.columns.values]
test_features.fillna(0, inplace=True)

Index(['event', 'key', 'time', 'duration'], dtype='object')
Index(['event', 'key', 'time', 'duration'], dtype='object')


In [3]:
X_train = train_features.drop(columns=['key_'])
y_train = train_data['label']
X_test = test_features.drop(columns=['key_'])
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [6]:
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 37
'y' sizes: 1423


In [10]:
import pandas as pd
import numpy as np

# Load the training and testing data
train_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\keystrokes.csv')
test_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\test1.csv')

# Verify column names
print(train_data.columns)
print(test_data.columns)

# Strip leading/trailing spaces from column names if necessary
train_data.columns = train_data.columns.str.strip()
test_data.columns = test_data.columns.str.strip()

# Add a new column named 'target' with the value 1 for all rows
train_data.insert(3, 'target', 1)
test_data.insert(3, 'target', 1)

# Assuming all data in train_data is from the owner (label=1) and test_data is from non-owner (label=0)
train_data['label'] = 1
test_data['label'] = 0

# Convert 'time' column to datetime, coercing errors
train_data['time'] = pd.to_datetime(train_data['time'], errors='coerce')
test_data['time'] = pd.to_datetime(test_data['time'], errors='coerce')

# Handle NaT values - you can either drop them or fill them with some value
train_data.dropna(subset=['time'], inplace=True)
test_data.dropna(subset=['time'], inplace=True)

# Ensure 'key' column exists after stripping spaces
if 'key' not in train_data.columns or 'key' not in test_data.columns:
    raise KeyError("The 'key' column is missing from the dataset after verifying column names.")

# Calculate additional features for training data
train_data['duration'] = train_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)

# Calculate additional features for testing data
test_data['duration'] = test_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)

# Preparing feature matrix X and target vector y
train_features = train_data.groupby('key').agg({
    'duration': ['mean', 'std'],
    'time': ['count']
}).reset_index()
train_features.columns = ['_'.join(col).strip() for col in train_features.columns.values]
train_features.fillna(0, inplace=True)

test_features = test_data.groupby('key').agg({
    'duration': ['mean', 'std'],
    'time': ['count']
}).reset_index()
test_features.columns = ['_'.join(col).strip() for col in test_features.columns.values]
test_features.fillna(0, inplace=True)

# Prepare feature matrix X and target vector y
X_train = train_features.drop(columns=['key_'])
y_train = train_data.drop_duplicates(subset='key')['label']
X_test = test_features.drop(columns=['key_'])
y_test = test_data.drop_duplicates(subset='key')['label']

# Ensure the sizes match
print("Size of X_train:", X_train.shape)
print("Size of y_train:", y_train.shape)
print("Size of X_test:", X_test.shape)
print("Size of y_test:", y_test.shape)

# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Define the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Index(['event', 'key', 'time', 'duration'], dtype='object')
Index(['event', 'key', 'time', 'duration'], dtype='object')
Size of X_train: (37, 3)
Size of y_train: (37,)
Size of X_test: (35, 3)
Size of y_test: (35,)
Epoch 1/50


c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.2919 - loss: 0.8823 - val_accuracy: 0.9429 - val_loss: 0.6418
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.3564 - loss: 0.7665 - val_accuracy: 0.4857 - val_loss: 0.6754
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2559 - loss: 0.8203 - val_accuracy: 0.2857 - val_loss: 0.7094
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.4806 - loss: 0.7355 - val_accuracy: 0.1714 - val_loss: 0.7432
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4910 - loss: 0.7037 - val_accuracy: 0.0000e+00 - val_loss: 0.7767
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5943 - loss: 0.6941 - val_accuracy: 0.0000e+00 - val_loss: 0.8096
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7649 - loss: 0.6008 - val_accuracy: 0.0000e+00 - val_loss: 0.8423
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8682 - loss: 0.5578 - val_accuracy: 0.0000e+00 - val_loss: 0

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load the training and testing data
train_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\keystrokes.csv')
test_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\test1.csv')

# Verify column names
print(train_data.columns)
print(test_data.columns)

# Strip leading/trailing spaces from column names if necessary
train_data.columns = train_data.columns.str.strip()
test_data.columns = test_data.columns.str.strip()

# Add 'label' column
train_data['label'] = 1  # Assuming all rows in train_data are from the owner
test_data['label'] = 0   # Assuming all rows in test_data are from non-owner

# Convert 'time' column to datetime, coercing errors
train_data['time'] = pd.to_datetime(train_data['time'], errors='coerce')
test_data['time'] = pd.to_datetime(test_data['time'], errors='coerce')

# Handle NaT values
train_data.dropna(subset=['time'], inplace=True)
test_data.dropna(subset=['time'], inplace=True)

# Calculate 'duration' for each key (time between keystrokes)
train_data['duration'] = train_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)
test_data['duration'] = test_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)

# Prepare feature matrix X and target vector y
X_train = train_data[['duration']]
y_train = train_data['label']

X_test = test_data[['duration']]
y_test = test_data['label']

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Index(['event', 'key', 'time', 'duration'], dtype='object')
Index(['event', 'key', 'time', 'duration'], dtype='object')
Epoch 1/50


c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9164 - loss: 0.6275 - val_accuracy: 0.0000e+00 - val_loss: 1.1613
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.2883 - val_accuracy: 0.0000e+00 - val_loss: 2.7845
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0626 - val_accuracy: 0.0000e+00 - val_loss: 4.4547
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0205 - val_accuracy: 0.0000e+00 - val_loss: 5.5939
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0097 - val_accuracy: 0.0000e+00 - val_loss: 6.4162
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.0000e+00 - val_loss: 7.0719
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.0000e+00 - val_loss: 7.5902
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Load the training and testing data
train_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\keystrokes.csv')
test_data = pd.read_csv('D:\\MLT assigments\\user_authentication\\user_12\\test1.csv')

# Strip leading/trailing spaces from column names if necessary
train_data.columns = train_data.columns.str.strip()
test_data.columns = test_data.columns.str.strip()

# Add 'label' column
train_data['label'] = 1  # Assuming all rows in train_data are from the owner
test_data['label'] = 0   # Assuming all rows in test_data are from non-owner

# Convert 'time' column to datetime, coercing errors
train_data['time'] = pd.to_datetime(train_data['time'], errors='coerce')
test_data['time'] = pd.to_datetime(test_data['time'], errors='coerce')

# Handle NaT values
train_data.dropna(subset=['time'], inplace=True)
test_data.dropna(subset=['time'], inplace=True)

# Calculate 'duration' for each key (time between keystrokes)
train_data['duration'] = train_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)
test_data['duration'] = test_data.groupby('key')['time'].diff().dt.total_seconds().fillna(0)

# Prepare feature matrix X and target vector y
X_train = train_data[['duration']]
y_train = train_data['label']

X_test = test_data[['duration']]
y_test = test_data['label']

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model with additional complexity
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on training and test data
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Train Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

# Inspect model predictions
predictions = model.predict(X_test)
predictions = np.round(predictions).astype(int).flatten()

# Print a classification report
print("Classification Report:")
print(classification_report(y_test, predictions))

# Print a sample of predictions vs true labels
print("Sample Predictions vs True Labels:")
for pred, true in zip(predictions[:10], y_test[:10]):
    print(f"Predicted: {pred}, True: {true}")
    


Epoch 1/50


c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7154 - loss: 0.6829 - val_accuracy: 0.0000e+00 - val_loss: 0.7262
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9685 - loss: 0.6489 - val_accuracy: 0.0000e+00 - val_loss: 0.7862
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9958 - loss: 0.5904 - val_accuracy: 0.0000e+00 - val_loss: 0.8736
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9929 - loss: 0.5262 - val_accuracy: 0.0000e+00 - val_loss: 1.0004
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9983 - loss: 0.4445 - val_accuracy: 0.0000e+00 - val_loss: 1.1884
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.3537 - val_accuracy: 0.0000e+00 - val_loss: 1.4599
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9970 - loss: 0.2727 - val_accuracy: 0.0000e+00 - val_loss: 1.8224
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.1919 - val_accuracy: 0

c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Tara Krishna\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn